In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import argparse
import matplotlib.pyplot as plt
import matplotlib
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, AvgPool2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array
import tensorflow as tf
import os
from keras import applications

In [ ]:
def plot_model_history(model_history):
    """
    Plot Accuracy and Loss curves given the model_history
    """
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    # summarize history for accuracy
    axs[0].plot(range(1, len(model_history.history['accuracy'])+1),
                model_history.history['accuracy'])
    axs[0].plot(range(1, len(model_history.history['val_accuracy'])+1),
                model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1, len(model_history.history['accuracy'])+1),
                      len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1, len(model_history.history['loss'])+1),
                model_history.history['loss'])
    axs[1].plot(range(1, len(model_history.history['val_loss'])+1),
                model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(
        1, len(model_history.history['loss'])+1), len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    fig.savefig('plot_test_model.png')
    plt.show()

In [ ]:
train_dir = '../input/faces-data/data/train'
val_dir = '../input/faces-data/data/test'

num_train = 28273
num_val = 7067
batch_size = 64
num_epoch = 40

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip = True, rotation_range = 15, zoom_range = [0.75,1])
val_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip = True, rotation_range = 15, zoom_range = [0.75,1])#ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical',)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=batch_size,
    color_mode="grayscale",
    class_mode='categorical', 
    shuffle = False)

In [ ]:
weights = {0:1.86, 1:1.86, 2:1, 3: 1.44, 4: 1.53, 5: 2.36}

In [ ]:
#RESNET
# trying to use resnet
img_height,img_width = 48,48
# base_model = applications.resnet50.ResNet50(weights= None, include_top=True, input_shape= (img_height,img_width,1), classes = 6)
VGG_model = applications.vgg16.VGG16(weights= None, include_top=True, input_shape= (img_height,img_width,1), classes = 6, classifier_activation="softmax")
adam = Adam(lr=0.0001)
VGG_model.compile(optimizer= adam, loss='categorical_crossentropy', metrics=['accuracy'])
VGG_history = VGG_model.fit(
        train_generator,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=validation_generator,
        validation_steps=num_val // batch_size,
        class_weight = weights)
plot_model_history(VGG_history)
VGG_model.save_weights('VGGweights.h5')
VGG_model.save('VGG_model.h5')
hist_df_VGG = pd.DataFrame(VGG_history.history) 
hist_csv_file = 'VGG_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df_VGG.to_csv(f)

In [ ]:
hist_df_resnet = pd.DataFrame(resnet_history.history) 
hist_csv_file = 'resnet_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df_resnet.to_csv(f)
resnet_model.save_weights('resnetweights.h5')
resnet_model.save('resnet_model.h5')

In [ ]:
from keras.layers import Conv2D, Activation, MaxPool2D, Dropout, Dense, BatchNormalization, Flatten
model = Sequential()
model.add(Conv2D(64, 5, data_format="channels_last", kernel_initializer="he_normal",
                 input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(64, 4))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.5))

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))

model.add(Conv2D(32, 3))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPool2D(pool_size=(2,2), strides=2))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.2))

model.add(Dense(6))
model.add(Activation("softmax"))

model.compile(optimizer='Nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model_info = model.fit(
    train_generator,
    steps_per_epoch=num_train // batch_size,
    epochs=num_epoch,
    validation_data=validation_generator,
    validation_steps=num_val // batch_size,
    class_weight = weights)
plot_model_history(model_info)
model.save('balanced_zoom5_40ep.h5')
model.save_weights('weights_balanced_zoom5_40ep.h5')

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
model_info = model.fit_generator(
        train_generator,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=validation_generator,
        validation_steps=num_val // batch_size,
        class_weight = weights)
plot_model_history(model_info)
model.save_weights('model.h5')

In [ ]:
model = Sequential()
model.add(Conv2D(filters=6, kernel_size=(5,5), padding='valid', input_shape=(48,48,1), activation='tanh'))
model.add(AvgPool2D(pool_size=(2,2)))
model.add(Conv2D(filters=16, kernel_size=(5,5), padding='valid', activation='tanh'))
model.add(AvgPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(120, activation='tanh'))
model.add(Dense(84, activation='tanh'))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
model_info = model.fit_generator(
        train_generator,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=validation_generator,
        validation_steps=num_val // batch_size,
        class_weight = weights)
plot_model_history(model_info)
model.save_weights('model.h5')

In [ ]:
# MY MODEL
model1 = Sequential()

model1.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model1.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Flatten())
model1.add(Dense(1024, activation='relu'))
model1.add(Dense(6, activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
model1.summary()

In [ ]:
model_info = model1.fit(
        train_generator,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=validation_generator,
        validation_steps=num_val // batch_size,
        class_weight = weights)
plot_model_history(model_info)
model1.save_weights('mymodelweights_rotate_zoom_flip.h5')
model1.save('my_model_rotate_zoom_flip.h5')

In [ ]:
import pandas as pd
hist_df = pd.DataFrame(model_info.history) 
hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
print(model_info.history['val_accuracy'])

In [ ]:
model2 = Sequential()
model2.add(Conv2D(input_shape=(48,48,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model2.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model2.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model2.add(Flatten())
model2.add(Dense(units=1000,activation="relu"))
model2.add(Dense(units=6, activation="softmax"))
model2.compile(loss='categorical_crossentropy',optimizer=Adam(lr=0.0001, decay=1e-6),metrics=['accuracy'])
model_info = model2.fit(
        train_generator,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=validation_generator,
        validation_steps=num_val // batch_size,
        class_weight = weights)
plot_model_history(model_info)
model.save_weights('model2.h5')

In [ ]:
model1.save_weights('mymodelweights_rotate_zoom_flip.h5')
model1.save('my_model_rotate_zoom_flip.h5')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
validation_generator.reset()
probabilities = model1.predict_generator(generator=validation_generator, steps = (num_val//batch_size) + 1)
y_true = validation_generator.classes
# print(np.shape(probabilities))
# print(np.shape(probabilities.argmax(axis=1)))
# print(np.shape(y_true))
cm = confusion_matrix(y_true, probabilities.argmax(axis=1))
cmap=plt.cm.Blues
fig = plt.figure(figsize=(6,6))
matplotlib.rcParams.update({'font.size': 16})
ax  = fig.add_subplot(111)
matrix = ax.imshow(cm, interpolation='nearest', cmap=cmap)
fig.colorbar(matrix) 
for i in range(0,6):
  for j in range(0,6):  
    if (cm[i,j] == 1463 ): 
       ax.text(j,i,cm[i,j],va='center',color='white', ha='center')
    else:
      ax.text(j,i,cm[i,j],va='center', ha='center')
    
# ax.set_title('Confusion Matrix')
labels = ['angry', 'fearful', 'happy', 'neutral','sad', 'surprised']
ticks = np.arange(len(labels))
ax.set_xticks(ticks)
ax.set_xticklabels(labels, rotation=45)
ax.set_yticks(ticks)
ax.set_yticklabels(labels)
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()

print(sum(probabilities.argmax(axis=1)==validation_generator.classes)/num_val)

In [ ]:
type(cm)
np.savetxt("confusion_matrix.csv", cm, delimiter=",")

In [ ]:
print(np.shape(y_pred))
print(y_pred)
print(validation_generator.labels)
print(np.shape(validation_generator.labels))
print(validation_generator)